In [ ]:
# TODO I am not sure if we need to initiate a SparkContext or if the notebook takes care of this?

# Alternative to Databricks display function.
import pandas as pd
pd.set_option('max_columns', None)
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").config("spark.sql.broadcastTimeout", "-1").appName('repairs-cleaning').getOrCreate()

In [ ]:
# Repairs file location and type
file_location = "repairs_dlo.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","
multiline = "true"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
      .option("inferSchema", infer_schema) \
      .option("header", first_row_is_header) \
      .option("multiLine", multiline) \
      .option("sep", delimiter) \
      .load(file_location)

In [ ]:
# convert column names to lower case and replace spaces with underscores
df2 = df.toDF(*[c.lower().replace(' ', '_') for c in df.columns])

In [ ]:
# convert timestamp column to a datetime field type
df2 = df2.withColumn('timestamp', F.to_timestamp("timestamp", "dd/MM/yyyy HH:mm:ss"))

In [ ]:
# # remove text from timestamp column

# def get_mean_timestamp(timestamp):
#     if len(timestamp) <16:
#         return '01/01/1970 00:00:00'


# # # convert to a UDF Function by passing in the function and the return type of function (string in this case)
# udf_get_mean_timestamp = F.udf(get_mean_timestamp, TimestampType())
# # apply function
# df2 = df2.withColumn("timestamp", udf_get_mean_timestamp("timestamp"))
# df2.limit(10).toPandas().head()

In [ ]:
# drop rows without a valid date
df2 = df2.na.drop(subset=["timestamp"])

# remove \n from address field
df = df.withColumn('address', F.col('Address of repair'))
df = df.withColumn('address', F.regexp_replace('address', '\n', ' '))

In [ ]:
df2.limit(10).toPandas().head()

In [ ]:
# convert name field to Title case
df2 = df2.withColumn('name_of_resident', F.initcap(F.col('name_of_resident')))

# add data source column
df2 = df2.withColumn('data_source', F.lit('DLO'))

# rename columns
df2 = df2.withColumnRenamed('name_of_resident', 'name_full') \
    .withColumnRenamed('job_description', 'description_of_work') \
    .withColumnRenamed('which_trade_needs_to_respond_to_repair?', 'trade_description') \
    .withColumnRenamed('what_is_the_priority_for_the_repair?', 'work_priority_description') \
    .withColumnRenamed('date_of_appointment', 'appointment_date') \
    .withColumnRenamed('if_there_is_a_cautionary_contact_alert,_what_is_the_nature_of_it?', 'alert_regarding_person_notes') \
    .withColumnRenamed('if_yes,_what_vulnerabilities_do_they_have?', 'vulnerability_notes') \
    .withColumnRenamed('postcode_of_property', 'postal_code_raw') \
    .withColumnRenamed('planners_to_allocate_to_operatives', 'operative') \
    .withColumnRenamed('does_the_resident_have_any_vulnerabilities?', 'vulnerability_flag') \
    .withColumnRenamed('is_there_a_cautionary_contact_alert_at_this_address?', 'alert_regarding_person') \
    .withColumnRenamed('planners_to_allocate_to_operatives', 'operative') \
    .withColumnRenamed('make_a_note_if_the_resident_is_reporting_any_coronavirus_symptoms_in_the_household_and_advise_residents_to_wear_a_face_mask_when_the_operative_is_in_the_property_and_to_maintain_social_distancing_', 'covid_notes') \
    .withColumnRenamed('have_you_read_the_coronavirus_statement_to_the_resident?_please_advise_the_resident_to_wear_a_face_mask_when_the_operative_is_in_the_property_and_to_maintain_social_distancing_', 'covid_statement_given') \
    .withColumnRenamed('uh_property_reference', 'property_reference_uh') \
    .withColumnRenamed('housing_status:_is_the_resident_a..._(select_as_many_as_apply)', 'property_address_type') \
    .withColumnRenamed('is_the_job_a_recharge_or_sus_recharge?', 'recharge') \
    .withColumnRenamed('form_reference_-_do_not_alter', 'form_ref') \
    .withColumnRenamed('phone_number_of_resident', 'phone_1') \
    .withColumnRenamed('address_of_repair', 'property_address') \
    .withColumnRenamed('phone_number_of_resident', 'phone_1') \
    .withColumnRenamed('planners_notes', 'notes') \
    .withColumnRenamed('time_of_appointment', 'appointment_time') \
    .withColumnRenamed('planners_notes', 'notes') \
    .withColumnRenamed('email_address', 'email_staff') \
    .withColumnRenamed('uh_phone_number_1', 'phone_2') \
    .withColumnRenamed('uh_phone_number_2', 'phone_3') \
    .withColumnRenamed('timestamp', 'datetime_raised') \


df2.toPandas().head()

In [ ]:
# df2.toPandas()['work_priority_description'].unique()

In [ ]:
# remap 'work_priority_priority_code' column
def map_repair_priority(code):
    if code == 'Immediate (2hr response)':
        return 1
    elif code == 'Emergency (24hrs)':
        return 2
    elif code == 'Urgent (5 working days)':
        return 3
    elif code == 'Normal (21 working days)':
        return 4
    else:
        return None

# # convert to a UDF Function by passing in the function and the return type of function (string in this case)
udf_map_repair_priority = F.udf(map_repair_priority, StringType())
# apply function
df2 = df2.withColumn('work_priority_code', udf_map_repair_priority('work_priority_description'))

In [ ]:
# get UPRN data
file_location = "vulnerability.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","
multiline = "true"

# The applied options are for CSV files. For other file types, these will be ignored.
vp = spark.read.format(file_type) \
      .option("inferSchema", infer_schema) \
      .option("header", first_row_is_header) \
      .option("multiLine", multiline) \
      .option("sep", delimiter) \
      .load(file_location)


In [ ]:
# keep certain columns
vp = vp.select('uprn', 'ten_property_ref')
# rename so has same name as repairs table
vp = vp.withColumnRenamed('ten_property_ref', 'property_reference_uh')

In [ ]:
# join uh prop ref to get uprn
df2 = df2.join(vp, 'property_reference_uh', 'left')

In [ ]:
# select and organise columns
df2 = df2.select(['id', 'datetime_raised',
 'operative',
 'notes',
 'name_full',
 'property_address', 
 'uprn',
 'phone_1',
 'property_address_type',
 'description_of_work',
 'trade_description',
 'work_priority_description',
 'work_priority_code',
 'appointment_date',
 'appointment_time',
 'covid_statement_given',
 'covid_notes',
 'recharge',
 'alert_regarding_person',
 'alert_regarding_person_notes',
 'vulnerability_flag',
 'vulnerability_notes',
 'email_staff',
 'postal_code_raw',
 'phone_2',
 'phone_3',
 'block_name',
 'estate_name',
 'block_reference',
 'estate_reference',
 'property_reference_uh',
 'form_ref',
 'data_source'
])

In [ ]:
df2.toPandas().sample(5)

In [ ]:
df2.toPandas().to_csv('repairs_dlo_cleaned.csv')